In [1]:
from keras.preprocessing import sequence
from keras.models import Sequential, Model, load_model, model_from_yaml
from keras.layers import Dense, Dropout, Activation
from keras.layers import Embedding
from keras.layers import Conv1D, GlobalMaxPooling1D
from keras.datasets import imdb
import tensorflow as tf
from keras import backend as K

tf.logging.set_verbosity(tf.logging.ERROR)
import numpy as np
import pandas as pd

from skater.core.local_interpretation.dnni.deep_interpreter import DeepInterpreter
from skater.core.visualizer.text_relevance_visualizer import build_visual_explainer, show_in_notebook
from skater.util.dataops import convert_dataframe_to_dict

/home/ubuntu/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Instructions for updating:
Use the retry module or similar alternatives.


In [2]:
# Create a TensorFlow session and register it with Keras. It will use this session to initialize all the variables
sess = tf.Session()
K.set_session(sess)

In [3]:
# set parameters
max_features = 20000
maxlen = 80
batch_size = 32
embedding_dims = 128
filters = 250
kernel_size = 3
hidden_dims = 250
epochs = 3

### Load the Dataset
#### IMDB dataset: 
##### 1. http://ai.stanford.edu/~amaas//data/sentiment/
##### 2. http://ai.stanford.edu/~ang/papers/acl11-WordVectorsSentimentAnalysis.pdf ( Section 4.1 )

In [4]:
# The Dataset contains 50,000 reviews(Train:25,000 and Test:25,000)
# More info about the dataset: https://keras.io/datasets/#imdb-movie-reviews-sentiment-classification

print('Loading data...')
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')

Loading data...
25000 train sequences
25000 test sequences


In [5]:
# https://stackoverflow.com/questions/42821330/restore-original-text-from-keras-s-imdb-dataset
# Reading raw text
INDEX_FROM = 3
# Get the {Word: Index} mapping
word_to_id = imdb.get_word_index()

def adjust_word_id_offset(word_id_dict):
    word_id_dict = {k:(v+INDEX_FROM) for k,v in word_id_dict.items()}
    word_id_dict["<PAD>"] = 0
    word_id_dict["<START>"] = 1
    word_id_dict["<UNK>"] = 2
    return word_id_dict

w_to_id = adjust_word_id_offset(word_to_id)

def get_raw_txt(word_id_dict, input_data):
    id_to_word = {value:key for key,value in word_id_dict.items()}
    return ' '.join([(id_to_word[_id] if _id in id_to_word else 'None') for _id in input_data])

r_t = get_raw_txt(w_to_id, x_train[20])
print(r_t + "\n")
print("Length: {}".format(len(r_t.split(' '))))

<START> shown in australia as <UNK> this incredibly bad movie is so bad that you become <UNK> and have to watch it to the end just to see if it could get any worse and it does the storyline is so predictable it seems written by a high school dramatics class the sets are pathetic but marginally better than the <UNK> and the acting is wooden br br the infant <UNK> seems to have been stolen from the props cupboard of <UNK> <UNK> there didn't seem to be a single original idea in the whole movie br br i found this movie to be so bad that i laughed most of the way through br br malcolm mcdowell should hang his head in shame he obviously needed the money

Length: 129


In [6]:
print('Pad sequences (samples x time)')
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

Pad sequences (samples x time)
x_train shape: (25000, 80)
x_test shape: (25000, 80)


In [7]:
# Raw text post selecting the top most frequently occurring words
index_train = 20
r_t_r = get_raw_txt(w_to_id, x_train[index_train])
print(r_t_r + "\n")
print("Length: {}".format(len(r_t_r.split(' '))))

dramatics class the sets are pathetic but marginally better than the <UNK> and the acting is wooden br br the infant <UNK> seems to have been stolen from the props cupboard of <UNK> <UNK> there didn't seem to be a single original idea in the whole movie br br i found this movie to be so bad that i laughed most of the way through br br malcolm mcdowell should hang his head in shame he obviously needed the money

Length: 80


In [8]:
# Reference: https://github.com/keras-team/keras/blob/master/examples/imdb_cnn.py
print('Build model...')
model = Sequential()

# we start off with an efficient embedding layer which maps
# our vocab indices into embedding_dims dimensions
model.add(Embedding(input_dim=max_features,
                    output_dim=embedding_dims,
                    input_length=maxlen))
model.add(Dropout(0.2))

# we add a Convolution1D, which will learn filters
# word group filters of size filter_length:
model.add(Conv1D(filters,
                 kernel_size,
                 padding='valid',
                 activation='relu',
                 strides=1))

# we use max pooling:
model.add(GlobalMaxPooling1D())

# We add a vanilla hidden layer:
model.add(Dense(hidden_dims))
model.add(Dropout(0.2))
model.add(Activation('relu'))

# We project onto a single unit output layer, and squash it with a sigmoid:
model.add(Dense(1))
model.add(Activation('sigmoid'))

Build model...


In [9]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [10]:
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=(x_test, y_test))

Train on 25000 samples, validate on 25000 samples
Epoch 1/3
25000/25000 [==============================] - 36s - loss: 0.4460 - acc: 0.7760 - val_loss: 0.3461 - val_acc: 0.8466
Epoch 2/3
25000/25000 [==============================] - 36s - loss: 0.2366 - acc: 0.9056 - val_loss: 0.3452 - val_acc: 0.8506
Epoch 3/3
25000/25000 [==============================] - 36s - loss: 0.0938 - acc: 0.9665 - val_loss: 0.5190 - val_acc: 0.8233


In [15]:
score, acc = model.evaluate(x_test, y_test,
                            batch_size=batch_size)
print('Test score:', score)
print('Test accuracy:', acc)

24768/25000 [============================>.] - ETA: 0sTest score: 0.47839841115951537
Test accuracy: 0.83748


#### Summarize the Model

In [16]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 80, 128)           2560000   
_________________________________________________________________
dropout_3 (Dropout)          (None, 80, 128)           0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 78, 250)           96250     
_________________________________________________________________
global_max_pooling1d_2 (Glob (None, 250)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 250)               62750     
_________________________________________________________________
dropout_4 (Dropout)          (None, 250)               0         
_________________________________________________________________
activation_3 (Activation)    (None, 250)               0         
__________

### Persist the model for future use

In [17]:
# Save and persist the trained keras model in YAML format
model_yaml = model.to_yaml()
with open("model_cnn_imdb_{}.yaml".format(epochs), "w") as yaml_file:
    yaml_file.write(model_yaml)
# serialize weights to HDF5
model.save_weights("model_cnn_imdb_{}.h5".format(epochs))
print("Save model to disk")

Save model to disk


### Load the saved model

In [10]:
# load the model
# Learning phase is set to '0' as we are not training
K.set_learning_phase(0)
yaml_file = open('model_cnn_imdb_{}.yaml'.format(epochs), 'r')
loaded_model_yaml = yaml_file.read()
yaml_file.close()
loaded_model = model_from_yaml(loaded_model_yaml)
# load weights into new model
loaded_model.load_weights('model_cnn_imdb_{}.h5'.format(epochs))
print("Load model from disk")


# Validate model performance with the reload of persisted model
loaded_model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
score, acc = loaded_model.evaluate(x_test, y_test,
                            batch_size=batch_size)
print("\n\n")
print('Test score:', score)
print('Test accuracy:', acc)

Load model from disk
24896/25000 [============================>.] - ETA: 0s


Test score: 0.5111530197954178
Test accuracy: 0.8468


### Lets ask Skater to help us in interpreting the model

In [11]:
index = 45
K.set_learning_phase(0)
with DeepInterpreter(session=K.get_session()) as di:
    print("learning_phase {}".format(K.learning_phase()))
    yaml_file = open('model_cnn_imdb_{}.yaml'.format(epochs), 'r')
    loaded_model_yaml = yaml_file.read()
    yaml_file.close()
    
    loaded_model = model_from_yaml(loaded_model_yaml)
    # load weights into new model
    loaded_model.load_weights('model_cnn_imdb_{}.h5'.format(epochs))
    print("Load model from disk")    
    
    # Input data
    xs = np.array([x_test[index]])
    ys = np.array([y_test[index]])

    print('Predicted class Probabilities: {}'.format(loaded_model.predict_proba(np.array([x_test[index]]))))
    print('Ground Truth: {}'.format(ys))
    
    embedding_tensor = loaded_model.layers[0].output
    input_tensor = loaded_model.layers[0].input
    
    embedding_out = di.session.run(embedding_tensor, {input_tensor: xs});
    # Using Integrated Gradient for computing feature relevance
    relevance_scores = di.explain('ig', loaded_model.layers[-2].output * ys, 
                                  loaded_model.layers[1].input, embedding_out, use_case='txt');

learning_phase 0
Load model from disk
1/1 [==============================] - 0s

2018-05-17 20:58:16,094 - IntegratedGradients - INFO - Executing operations to compute relevance using Integrated Gradient



Predicted class Probabilities: [[0.8022024]]
Ground Truth: [1]


In [12]:
# building a dataframe with columns 'features' and 'relevance scores'
# Since, the relevance score is compute over the embedding vector, we aggregate it by computing 'mean'
# over the embedding to get scalar coefficient for the features
relevance_scores_df = pd.DataFrame(relevance_scores[0]).mean(axis=1)
relevance_scores_df.describe()

count    80.000000
mean      0.000129
std       0.008240
min      -0.043927
25%      -0.001377
50%       0.000383
75%       0.002411
max       0.032541
dtype: float64

In [13]:
# # Retrieve the text
r_t_test = get_raw_txt(w_to_id, x_test[index])
print(r_t_test)

excellent tale of two boys that do whatever they can to get away from there abusive <UNK> father lord of the rings star elijah wood is outstanding in this unforgettable role this movie is one of the main reasons i haven't touched a single beer and never will as long as i live that might make me sound like a nerd but that's what i have to say it is a wonder why this isn't as a classic american tale


#### Visualize the results

In [14]:
build_visual_explainer(r_t_test, relevance_scores_df, highlight_oov=True, enable_plot=True)

2018-05-17 20:58:16,793 - skater.core.visualizer.text_relevance_visualizer - INFO - Rank order feature relevance based on input created and saved as feature_relevance.png
2018-05-17 20:58:16,795 - skater.core.visualizer.text_relevance_visualizer - INFO - Relevance plot name: feature_relevance.png
2018-05-17 20:58:16,827 - skater.core.visualizer.text_relevance_visualizer - INFO - Visual Explainer built, use show_in_notebook to render in Jupyter style Notebooks: rendered.html


In [15]:
show_in_notebook('./rendered.html')

2018-05-17 20:58:16,833 - skater.core.visualizer.text_relevance_visualizer - INFO - File Name: ./rendered.html


### Generating conditional adversarial examples

#### Generating text by deleting or updating textual information

In [15]:
new_txt = r_t_test.replace("outstanding", "<UNK>")
print(new_txt)

excellent tale of two boys that do whatever they can to get away from there abusive <UNK> father lord of the rings star elijah wood is <UNK> in this unforgettable role this movie is one of the main reasons i haven't touched a single beer and never will as long as i live that might make me sound like a nerd but that's what i have to say it is a wonder why this isn't as a classic american tale


In [16]:
# Convert the dataset to engineered feature format
# Reference: https://stackoverflow.com/questions/42964375/how-to-input-new-text-for-prediction-in-keras-while-using-an-inbuilt-dataset

def input_formatter_imdb(input_txt, word_index_mapping):
    
    x_i_test = [[word_index_mapping[wrds] if wrds in word_index_mapping.keys() else word_index_mapping["<UNK>"] 
                 for wrds in input_txt.split(' ')]]
    x_i_test = sequence.pad_sequences(x_i_test, maxlen=maxlen)
    txt_vector = np.array([x_i_test.flatten()])
    return txt_vector

In [17]:
input_vector = input_formatter_imdb(new_txt, w_to_id)
input_vector

array([[  321,   787,     7,   107,  1013,    15,    81,   845,    36,
           70,     8,    79,   245,    39,    50,  4595,     2,   336,
         1635,     7,     4,  2666,   323, 13245,  2137,     9,     2,
           11,    14,  3210,   217,    14,    20,     9,    31,     7,
            4,   293,  1007,    13,   774,  2842,     6,   686,  3640,
            5,   115,    80,    17,   196,    17,    13,   412,    15,
          238,    97,    72,   481,    40,     6,  5155,    21,   198,
           51,    13,    28,     8,   135,    12,     9,     6,   594,
          138,    14,   218,    17,     6,   356,   298,   787]],
      dtype=int32)

In [18]:
K.set_learning_phase(0)
with DeepInterpreter(session=K.get_session()) as di:
    print("learning_phase {}".format(K.learning_phase()))
    yaml_file = open('model_cnn_imdb_{}.yaml'.format(epochs), 'r')
    loaded_model_yaml = yaml_file.read()
    yaml_file.close()
    
    loaded_model = model_from_yaml(loaded_model_yaml)
    # load weights into new model
    loaded_model.load_weights('model_cnn_imdb_{}.h5'.format(epochs))
    print("Load model from disk")    
    
    # Input data
    xs = input_vector
    ys = np.array([1])

    print('Predicted class : {}'.format(loaded_model.predict_proba(np.array([x_test[index]]))))
    print('Ground Truth: {}'.format(ys))
    
    embedding_tensor = loaded_model.layers[0].output
    input_tensor = loaded_model.layers[0].input
    
    embedding_out = di.session.run(embedding_tensor, {input_tensor: xs})
    print(embedding_out.shape)
    # Using Integrated Gradient for computing feature relevance
    relevance_scores = di.explain('ig', loaded_model.layers[-2].output * ys, 
                                  loaded_model.layers[1].input, embedding_out, use_case='txt');

learning_phase 0
Load model from disk
1/1 [==============================] - 0s

2018-04-29 17:37:07,779 - IntegratedGradients - INFO - Executing operations to compute relevance using Integrated Gradient



Predicted class : [[0.8917762]]
Ground Truth: [1]
(1, 80, 128)


In [19]:
relevance_scores_df = pd.DataFrame(relevance_scores[0]).mean(axis=1)
relevance_scores_df.describe()

# Retrieve the text
_in = input_vector.reshape(-1)
r_t = get_raw_txt(w_to_id, _in)
print("New Document:\n\n {}".format(r_t))

build_visual_explainer(r_t, relevance_scores_df, highlight_oov=True, enable_plot=True)
show_in_notebook('./rendered.html')

New Document:

 excellent tale of two boys that do whatever they can to get away from there abusive <UNK> father lord of the rings star elijah wood is <UNK> in this unforgettable role this movie is one of the main reasons i haven't touched a single beer and never will as long as i live that might make me sound like a nerd but that's what i have to say it is a wonder why this isn't as a classic american tale


2018-04-29 17:37:08,269 - skater.core.visualizer.text_relevance_visualizer - INFO - Rank order feature relevance based on input created and saved as feature_relevance.png
2018-04-29 17:37:08,270 - skater.core.visualizer.text_relevance_visualizer - INFO - Relevance plot name: feature_relevance.png
2018-04-29 17:37:08,295 - skater.core.visualizer.text_relevance_visualizer - INFO - Visual Explainer built, use show_in_notebook to render in Jupyter style Notebooks: rendered.html


In [20]:
# replacing words in a sentence
import re
words_to_replace = ["excellent", 'classic', 'unforgettable', 'touched', 'never']
regex_builder = re.compile('|'.join(map(re.escape, words_to_replace)))
new_txt2 = regex_builder.sub("<UNK>" , new_txt)
print(new_txt2)

<UNK> tale of two boys that do whatever they can to get away from there abusive <UNK> father lord of the rings star elijah wood is <UNK> in this <UNK> role this movie is one of the main reasons i haven't <UNK> a single beer and <UNK> will as long as i live that might make me sound like a nerd but that's what i have to say it is a wonder why this isn't as a <UNK> american tale


In [21]:
input_vector2 = input_formatter_imdb(new_txt2, w_to_id)

In [22]:
K.set_learning_phase(0)
with DeepInterpreter(session=K.get_session()) as di:
    print("learning_phase {}".format(K.learning_phase()))
    yaml_file = open('model_cnn_imdb_{}.yaml'.format(epochs), 'r')
    loaded_model_yaml = yaml_file.read()
    yaml_file.close()
    
    loaded_model = model_from_yaml(loaded_model_yaml)
    # load weights into new model
    loaded_model.load_weights('model_cnn_imdb_{}.h5'.format(epochs))
    print("Load model from disk")    
    
    # Input data
    xs = input_vector2
    ys = np.array([1])

    print('Predicted class : {}'.format(loaded_model.predict_proba(np.array([x_test[index]]))))
    print('Ground Truth: {}'.format(ys))
    
    embedding_tensor = loaded_model.layers[0].output
    input_tensor = loaded_model.layers[0].input
    
    embedding_out = di.session.run(embedding_tensor, {input_tensor: xs})
    print(embedding_out.shape)
    # Using Integrated Gradient for computing feature relevance
    relevance_scores = di.explain('ig', loaded_model.layers[-2].output * ys, 
                                  loaded_model.layers[1].input, embedding_out, use_case='txt');

learning_phase 0
Load model from disk
1/1 [==============================] - 0s

2018-04-29 17:37:08,780 - IntegratedGradients - INFO - Executing operations to compute relevance using Integrated Gradient



Predicted class : [[0.8917762]]
Ground Truth: [1]
(1, 80, 128)


In [23]:
relevance_scores_df = pd.DataFrame(relevance_scores[0]).mean(axis=1)
print(relevance_scores_df.describe())

# Retrieve the text
_in = input_vector.reshape(-1)
r_t = get_raw_txt(w_to_id, _in)
print("New Document:\n\n {}".format(r_t))

build_visual_explainer(r_t, relevance_scores_df, highlight_oov=True, enable_plot=True)
show_in_notebook('./rendered.html')

count    80.000000
mean     -0.000345
std       0.005353
min      -0.028120
25%      -0.001197
50%       0.000094
75%       0.001343
max       0.013182
dtype: float64
New Document:

 excellent tale of two boys that do whatever they can to get away from there abusive <UNK> father lord of the rings star elijah wood is <UNK> in this unforgettable role this movie is one of the main reasons i haven't touched a single beer and never will as long as i live that might make me sound like a nerd but that's what i have to say it is a wonder why this isn't as a classic american tale


2018-04-29 17:37:09,421 - skater.core.visualizer.text_relevance_visualizer - INFO - Rank order feature relevance based on input created and saved as feature_relevance.png
2018-04-29 17:37:09,422 - skater.core.visualizer.text_relevance_visualizer - INFO - Relevance plot name: feature_relevance.png
2018-04-29 17:37:09,445 - skater.core.visualizer.text_relevance_visualizer - INFO - Visual Explainer built, use show_in_notebook to render in Jupyter style Notebooks: rendered.html
